In [21]:
from __future__ import unicode_literals, division

In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
from embedding import get_embedding
import re
from config import Config
import sys

In [46]:
%load_ext autoreload
%autoreload 2

# Get configurations

In [3]:
config = Config()
max_len = config.max_sent_len
embedding_size = config.embedding_size

In [4]:
path = '../train.csv'

In [7]:
data = pd.read_csv(path, nrows=50000)
data = data[:40000]


In [8]:
columns = data.columns

In [11]:
columns

Index([u'id', u'qid1', u'qid2', u'question1', u'question2', u'is_duplicate'], dtype='object')

In [10]:
data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [15]:
data.isna().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

# sequence lengths

In [33]:
data['seql_one'] = data['question1'].apply(lambda row: len(str(row).split()))
data['seql_two'] = data['question2'].apply(lambda row: len(str(row).split()))

In [34]:
data.columns

Index([          u'id',         u'qid1',         u'qid2',    u'question1',
          u'question2', u'is_duplicate',     u'seql_one',     u'seql_two'],
      dtype='object')

# spliting data to train and dev

80% train data and 20% test data

In [35]:
split_ratio = 0.8

In [36]:
l = len(data)

In [37]:
train_length = int(l*split_ratio)

In [38]:
data.groupby('is_duplicate').count()

,id,qid1,qid2,question1,question2,seql_one,seql_two
is_duplicate,,,,,,,
0,25109,25109,25109,25109,25109,25109,25109
1,14891,14891,14891,14891,14891,14891,14891


# word2vec features


In [39]:
def get_sent_embedding(sent):
    sent = str(sent).lower().strip()
    vector = []
    counter = 0
    vector = [get_embedding(w) for w in sent.split()]
    return vector

In [31]:
# get_sent_embedding('dsfhgdsj sajkfgksdgf')

In [40]:
len(get_sent_embedding('i love reading'))

3

In [41]:
train_data = data[:train_length]


train_data['vec_1'] = train_data['question1'].apply(lambda row: get_sent_embedding(row))
train_data['vec_2'] = train_data['question2'].apply(lambda row: get_sent_embedding(row))

train_data = zip(train_data.vec_1, train_data.vec_2, train_data.seql_one, train_data.seql_two, train_data.is_duplicate)


In [42]:
dev_data = data[train_length:]


dev_data['vec_1'] = dev_data['question1'].apply(lambda row: get_sent_embedding(row))
dev_data['vec_2'] = dev_data['question2'].apply(lambda row: get_sent_embedding(row))

dev_data = zip(dev_data.vec_1, dev_data.vec_2, dev_data.seql_one, dev_data.seql_two, dev_data.is_duplicate)


In [60]:
dev_data[0][4]

1

# Building and training Model

In [106]:
from model import Model
model = Model()

In [107]:
tf.reset_default_graph()
model.build()

In [108]:
model.train(train_data=train_data, dev_data=dev_data)

KeyboardInterrupt: 

Output from first three epochs:

At epoch 0 loss is..0.591495674435
At epoch 0 dev acc..0.660000026226
---------------------------------------
At epoch 1 loss is..0.515076757554
At epoch 1 dev acc..0.839999973774
---------------------------------------
At epoch 2 loss is..0.455765611057
At epoch 2 dev acc..0.819999992847
---------------------------------------


In [76]:
[0]*8

[0, 0, 0, 0, 0, 0, 0, 0]